# ou22 Powell

In [3]:
import pandas as pd
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
from scipy.optimize import minimize
import scipy
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.1/Resources/'
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import datetime
from rpy2.robjects.vectors import FloatVector
import multiprocessing

np.random.seed(22101)

# Define the model that generates pair simulations.
yuima = importr("yuima")
n_jumpou_NIG_sim_string = """
n_jumpou_NIG = function(random_seed, num_sim,
                        mu11, mu12, mu21, mu22, 
                        sigma11, sigma12, sigma21, sigma22,
                        j11, j12, j21, j22,
                        alpha, beta1, beta2, delta0, mu1, mu2, 
                        lambda11, lambda12, lambda21, lambda22,
                        xinit_vec, T0, T, length){
  
  set.seed(random_seed)
  
  drift = c("mu11-mu12*X1", "mu21-mu22*X2")
  diffusion = matrix(c("exp(sigma11)", "exp(sigma12)", "exp(sigma21)", "exp(sigma22)"), 2, 2, byrow=TRUE)
  jumpcoef = matrix(c("j11", "j12", "j21", "j22"), 2, 2, byrow=TRUE) 
  
  alpha = alpha
  beta = c(beta1, beta2)
  delta0 = delta0
  mu = c(mu1, mu2)
  Lambda = matrix(c(lambda11, lambda12, lambda21, lambda22), 2, 2, byrow=TRUE)
  
  ou_model = setModel(drift=drift, diffusion=diffusion, jump.coeff=jumpcoef, 
                      measure.type="code",
                      measure=list(df="rNIG(z, alpha, beta, delta0, mu, Lambda)"), 
                      time.variable = "t",
                      state.var=c("X1","X2"), solve.variable=c("X1","X2"))
  newsamp = setSampling(Initial=T0, Terminal=T, n=length)
  
  n_sim_data = data.frame(matrix(nrow=length+1, ncol=2*num_sim))
  for (i in 1:num_sim){
    jumpou_sim = simulate(ou_model, 
                          true.par=list(
                            mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22, 
                            sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22,
                            j11=j11, j12=j12, j21=j21, j22=j22,
                            alpha=alpha, beta=beta, delta0=delta0, mu=mu, Lambda=Lambda), 
                          xinit=xinit_vec[[i]], sampling=newsamp)
    original_data = jumpou_sim@data@original.data
    one_sim_jumpou = data.frame(original_data[,1], original_data[,2])
    colnames(one_sim_jumpou) = c('series1', 'series2')
    n_sim_data[, (2*i-1):(2*i)] = one_sim_jumpou
  }
  return(n_sim_data)
}

n_pair_prices = read.csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Generator/sp500_20180101_20181231_pair_prices.csv", row.names=1)
n_log_pair_prices = log(n_pair_prices)
xinit_vec = list()
for (i in 1:(dim(n_pair_prices)[2]/2)){
  xinit_vec[i] = as.numeric(n_log_pair_prices[1, (2*i-1):(2*i)])
}
"""

n_jumpou_NIG_sim = SignatureTranslatedAnonymousPackage(n_jumpou_NIG_sim_string, "n_jumpou_NIG_sim")
def n_jumpou_simulation(random_seed, num_sim,
                        mu11, mu12, mu21, mu22, 
                        sigma11, sigma12, sigma21, sigma22,
                        j11, j12, j21, j22,
                        alpha, beta1, beta2, delta0, mu1, mu2, 
                        lambda11, lambda12, lambda21, lambda22,
                        xinit_vec, T0, T, length):
    """num_sim simulations of bivariate Ornstein-Uhlenbeck process,
    length = length of one series
    """
    n_sim_data = pd.DataFrame(
        n_jumpou_NIG_sim.n_jumpou_NIG(random_seed, num_sim,
                                      mu11, mu12, mu21, mu22, 
                                      sigma11, sigma12, sigma21, sigma22,
                                      j11, j12, j21, j22,
                                      alpha, beta1, beta2, delta0, mu1, mu2, 
                                      lambda11, lambda12, lambda21, lambda22,
                                      xinit_vec, T0, T, length)).transpose()
    return n_sim_data


def price_to_log_price(n_price):
    return(np.log(n_price))

def log_price_to_price(n_log_price):
    return(np.exp(n_log_price))

def price_to_return(n_price):
    n_return = pd.DataFrame()
    for i in range(n_price.shape[1]):
        ith_column_price_series = n_price.iloc[:, i]
        n_return = pd.concat([n_return, 100 * (np.log(ith_column_price_series[1:].values) - np.log(ith_column_price_series[:-1]))], axis=1)
    return n_return

def log_price_to_return(n_log_price):
    n_real_return = pd.DataFrame()
    for i in range(n_log_price.shape[1]):
        ith_column_price_series = n_log_price.iloc[:, i]
        n_real_return = pd.concat([n_real_return, 100 * (ith_column_price_series[1:].values - ith_column_price_series[:-1])], axis=1)
    return n_real_return

    
def cal_stats(n_return, n_price=None):
    # (different expressions of calculation from intro to stat finance)
    # 4 statistics
    
    return_series1 = n_return.iloc[:, ::2]
    return_series2 = n_return.iloc[:, 1::2]
    
    
    mean1 = return_series1.mean(axis=0).values
    sd1 = return_series1.std(axis=0).values
    skew1 = return_series1.skew(axis=0).values
    kurtosis1 = return_series1.kurtosis(axis=0).values
    autocorr1 = return_series1.apply(lambda x: x.autocorr(lag=1))    
    
    
    mean2 = return_series2.mean(axis=0).values
    sd2 = return_series2.std(axis=0).values
    skew2 = return_series1.skew(axis=0).values
    kurtosis2 = return_series2.kurtosis(axis=0).values
    autocorr2 = return_series2.apply(lambda x: x.autocorr(lag=1)) 
    
    
    stats_data = pd.DataFrame([mean1, mean2, sd1, sd2, skew1, skew2, kurtosis1, kurtosis2, autocorr1, autocorr2])
    stats_data = stats_data.transpose()
    stats_data.columns = [
        'return_mean1', 'return_mean2',
        'return_sd1', 'return_sd2',
        'return_skew1', 'return_skew2',
        'return_kurtosis1', 'return_kurtosis2',
        'return_autocorrelation1', 'return_autocorrelation2']
    return stats_data



def loss_function(params):
    """n_real_stats is a global amount calculated outside the function"""
    params = FloatVector(params)
    print(params)
    moment_loss = pd.DataFrame().reindex_like(real_stats)
    
    n_sim_log_price = n_jumpou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=params[0], mu12=params[1], mu21=params[2], mu22=params[3], 
        sigma11=params[4], sigma12=params[5], sigma21=params[6], sigma22=params[7],
        j11=params[8], j12=params[9], j21=params[10], j22=params[11],
        alpha=params[12], beta1=params[13], beta2=params[14], delta0=params[15], mu1=params[16], mu2=params[17], 
        lambda11=params[18], lambda12=lambda12, lambda21=lambda21, lambda22=params[19],
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    
    print(n_sim_log_price.iloc[5, 5])
    n_sim_price = log_price_to_price(n_sim_log_price)
    n_sim_return = price_to_return(n_sim_price)
    n_sim_stats = cal_stats(n_sim_return)


    for i in range(n_real_stats.shape[0]):
        for j in range(n_real_stats.shape[1]):
            moment_loss.iloc[i, j] = np.sqrt((n_real_stats.iloc[i, j] - n_sim_stats.iloc[i, j])**2)
    sum_all = np.sum(moment_loss)
    print(sum_all)
    print(np.sum(sum_all))
    print('----------')

    return np.sum(sum_all)



real_price = pd.read_csv("sp500_20180101_20181231_pair_prices.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv("sp500_20180101_20181231_pair_returns.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=None)


xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
    
num_sim, T0, T, length = real_stats.shape[0], 0, 1, real_price.shape[0]

n_real_stats = real_stats







lambda12 = 0
lambda21 = 0








initial0 = [1, 1, 1, 1,
            1, 1, 1, 1,
            1, 1, 1, 1,
            2, 1, 1, 1, 1, 1, 
            1, 1]

begin_time = datetime.datetime.now()
res = minimize(loss_function, initial0, method='Powell',
               tol=1e-6, options={'disp': True},
               bounds=[(0., None), (0., None), (0., None), (0., None),
                       (None, None), (None, None), (None, None), (None, None),
                       (None, None), (None, None), (None, None), (None, None),
                       (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),
                       (None, None), (None, None),])
print(res.x)

time = datetime.datetime.now() - begin_time
print(time)

params = (res.x)
loss = loss_function((params))
print(loss)


 [1] 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1

5.432558477451675
return_mean1                205.584156
return_mean2                208.974733
return_sd1                 6029.982501
return_sd2                 6016.018832
return_skew1                271.376191
return_skew2                271.376191
return_kurtosis1           1718.628957
return_kurtosis2           2262.707975
return_autocorrelation1      16.590279
return_autocorrelation2      17.194542
dtype: float64
17018.43435864746
----------
 [1] 1.115092 1.000000 1.000000 1.000000 1.000000 1.000000 1.000000 1.000000
 [9] 1.000000 1.000000 1.000000 1.000000 2.000000 1.000000 1.000000 1.000000
[17] 1.000000 1.000000 1.000000 1.000000

4.9119361975463605
return_mean1                214.498558
return_mean2                209.302672
return_sd1                 6079.329311
return_sd2                 6065.493498
return_skew1                299.203249
return_skew2                299.203249
return_kurtosis1           2133.401383
return_kurtosi

4.674104262669337
return_mean1                209.178417
return_mean2                216.025066
return_sd1                 6156.514091
return_sd2                 6142.690891
return_skew1                320.797963
return_skew2                320.797963
return_kurtosis1           2266.241905
return_kurtosis2           2857.407812
return_autocorrelation1      18.127645
return_autocorrelation2      18.380431
dtype: float64
18526.16218612367
----------
 [1] 0.5293336 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
 [8] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 2.0000000 1.0000000
[15] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000

5.852939274146419
return_mean1                228.152351
return_mean2                228.544747
return_sd1                 6186.606029
return_sd2                 6172.684636
return_skew1                305.678259
return_skew2                305.678259
return_kurtosis1           2341.306520
return_kurtosis2           2760.103758
re

5.334662655952461
return_mean1                215.584738
return_mean2                218.159065
return_sd1                 6073.526512
return_sd2                 6059.594128
return_skew1                291.926642
return_skew2                291.926642
return_kurtosis1           2252.988886
return_kurtosis2           2632.904295
return_autocorrelation1      17.152065
return_autocorrelation2      18.052300
dtype: float64
18071.815271807067
----------
 [1] 0.5272171 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
 [8] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 2.0000000 1.0000000
[15] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000

4.8694691505156955
return_mean1                226.650348
return_mean2                224.444836
return_sd1                 6026.653986
return_sd2                 6012.759880
return_skew1                261.082578
return_skew2                261.082578
return_kurtosis1           1721.319843
return_kurtosis2           2150.966407


4.4024856654374735
return_mean1                239.312432
return_mean2                243.358812
return_sd1                 6102.257686
return_sd2                 6079.758121
return_skew1                272.109209
return_skew2                272.109209
return_kurtosis1           1954.018991
return_kurtosis2           2394.338498
return_autocorrelation1      17.373679
return_autocorrelation2      17.560081
dtype: float64
17592.196717906296
----------
 [1] 0.5272218 1.8372854 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
 [8] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 2.0000000 1.0000000
[15] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000

5.08027753117367
return_mean1                225.043691
return_mean2                216.830574
return_sd1                 6078.539287
return_sd2                 6056.408235
return_skew1                288.518600
return_skew2                288.518600
return_kurtosis1           2009.865030
return_kurtosis2           2461.810381
r

5.771458057180308
return_mean1                226.434837
return_mean2                232.583656
return_sd1                 6109.793200
return_sd2                 6087.391835
return_skew1                294.331550
return_skew2                294.331550
return_kurtosis1           2073.217558
return_kurtosis2           2623.204608
return_autocorrelation1      18.143934
return_autocorrelation2      17.638920
dtype: float64
17977.07164639657
----------
 [1] 0.5272218 1.8167641 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
 [8] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 2.0000000 1.0000000
[15] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000

4.327973089117856
return_mean1                231.081530
return_mean2                223.359144
return_sd1                 6179.675875
return_sd2                 6157.704021
return_skew1                329.174591
return_skew2                329.174591
return_kurtosis1           2375.370192
return_kurtosis2           2803.827554
re

...

4.52278304260908
return_mean1                290.975927
return_mean2                232.701249
return_sd1                 4521.842217
return_sd2                 3846.810392
return_skew1                165.803117
return_skew2                165.803117
return_kurtosis1           1094.751426
return_kurtosis2           1711.858577
return_autocorrelation1      17.513395
return_autocorrelation2      17.716391
dtype: float64
12065.775807359845
----------
 [1]  0.5272218  1.8167920  1.7249368  1.7442531  0.8119589  0.7623146
 [7]  1.0000000 -7.4832925 -0.3879134  0.9559237 -0.1759433  0.3666448
[13]  3.7114316  1.0000000  1.0000000  1.0000000  1.0000000  1.0000000
[19]  1.0000000  1.0000000

5.232542032618456
return_mean1                293.711688
return_mean2                236.439974
return_sd1                 4561.665159
return_sd2                 3850.834993
return_skew1                158.787259
return_skew2                158.787259
return_kurtosis1           1093.499093
return_kurtosis2

4.663616957456961
return_mean1                334.237283
return_mean2                276.179856
return_sd1                 4535.802382
return_sd2                 3844.836343
return_skew1                167.500185
return_skew2                167.500185
return_kurtosis1           1102.433606
return_kurtosis2           1711.510260
return_autocorrelation1      17.892853
return_autocorrelation2      17.330036
dtype: float64
12175.222988595367
----------
 [1]  0.5272218  1.8167920  1.7249368  1.7442531  0.8119589  0.7623146
 [7]  1.0000000 -7.4832925 -0.3879134  0.9559237 -0.1759433  0.3666448
[13]  3.6757569  2.0000000  1.0000000  1.0000000  1.0000000  1.0000000
[19]  1.0000000  1.0000000

5.38000727036506


R[write to console]: Error in rNIG(n, alpha, beta, delta0 * delta, mu * delta, Lambda) : 
  alpha^2 - t(beta) %*% Lambda %*% beta must be nonnegative.

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



return_mean1                321.161170
return_mean2                259.900237
return_sd1                 4537.686317
return_sd2                 3829.247506
return_skew1                169.182767
return_skew2                169.182767
return_kurtosis1           1123.927403
return_kurtosis2           1713.802626
return_autocorrelation1      17.847391
return_autocorrelation2      18.592977
dtype: float64
12160.53116114585
----------
 [1]  0.5272218  1.8167920  1.7249368  1.7442531  0.8119589  0.7623146
 [7]  1.0000000 -7.4832925 -0.3879134  0.9559237 -0.1759433  0.3666448
[13]  3.6757569  3.6180340  1.0000000  1.0000000  1.0000000  1.0000000
[19]  1.0000000  1.0000000



RRuntimeError: Error in rNIG(n, alpha, beta, delta0 * delta, mu * delta, Lambda) : 
  alpha^2 - t(beta) %*% Lambda %*% beta must be nonnegative.
